# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [33]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'epa', 'local', 'met', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [42]:
# review the collections in our new database
pprint(db.list_collection_names())

['establishments']


In [43]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64239cb15e6e664c5b59ccaa'),
 'geocode': {'latitude': 51.086058, 'longitude': 1.196408},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extr

In [9]:
# assign the collection to a variable
establishments = db.establishments

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [10]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [11]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

In [12]:
# Check that the new restaurant was inserted
query = {"BusinessName":"Penang Flavours"}
results = establishments.find(query)
for result in results:
    print(result)

{'_id': ObjectId('6423a42c622c349465d7b3e5'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': '', 'AddressLine1': 'Penang Flavours', 'AddressLine2': '146A Plumstead Rd', 'AddressLine3': 'London', 'AddressLine4': '', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [23]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = {'BusinessTypeID':1, 'BusinessType':1}
required_ID = establishments.find_one(query, fields)
print(required_ID)

{'_id': ObjectId('64239cb15e6e664c5b59c9c4'), 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [24]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
    {"BusinessName":"Penang Flavours"},
    {'$set':
     {'BusinessTypeID': required_ID['BusinessTypeID']}
    }
)

In [25]:
# Confirm that the new restaurant was updated
query = {"BusinessName":"Penang Flavours"}
fields = {'BusinessTypeID':1}
results = establishments.find(query, fields)
for result in results:
    print(result)

{'_id': ObjectId('6423a42c622c349465d7b3e5'), 'BusinessTypeID': 1}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [26]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName' : 'Dover'}
docs_dover = establishments.count_documents(query)
print(f"Number of establishments in Dover: {docs_dover}")

Number of establishments in Dover: 994


In [27]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many(query)

In [28]:
# Check if any remaining documents include Dover
docs_dover_recheck = establishments.count_documents(query)
print(f"Number of establishments in Dover: {docs_dover_recheck}")

Number of establishments in Dover: 0


In [34]:
# Check that other documents remain with 'find_one'
check = establishments.find_one()
pprint(check)

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64239cb15e6e664c5b59ccaa'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          '

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [38]:
# Change the data type from String to Decimal for longitude
establishments.update_many({}, [ {'$set':{ "geocode.longitude" : {'$toDouble': "$geocode.longitude"}}} ])

In [39]:
# Change the data type from String to Decimal for latitude
establishments.update_many({}, [ {'$set':{ "geocode.latitude" : {'$toDouble': "$geocode.latitude"}}} ])

In [41]:
# Check that the coordinates are now numbers
query = {}
fields = {'BusinessName': 1, 'geocode.longitude': 1, 'geocode.latitude': 1}

# Capture the results to a variable
check_results = establishments.find(query, fields)

# Pretty print the results
for each_result in check_results:
    pprint(each_result)

{'BusinessName': 'Wear Bay Bowls Club',
 '_id': ObjectId('64239cb15e6e664c5b59ccaa'),
 'geocode': {'latitude': 51.086058, 'longitude': 1.196408}}
{'BusinessName': 'St Marys COE (aided) Primary School',
 '_id': ObjectId('64239cb15e6e664c5b59ccac'),
 'geocode': {'latitude': 51.085797, 'longitude': 1.194762}}
{'BusinessName': 'Mariner',
 '_id': ObjectId('64239cb15e6e664c5b59ccae'),
 'geocode': {'latitude': 51.08084, 'longitude': 1.188537}}
{'BusinessName': 'The Ships Galley',
 '_id': ObjectId('64239cb15e6e664c5b59ccaf'),
 'geocode': {'latitude': 51.08084, 'longitude': 1.188537}}
{'BusinessName': 'The Pavilion',
 '_id': ObjectId('64239cb15e6e664c5b59ccb0'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625}}
{'BusinessName': 'Folkestone Trawlers Shop',
 '_id': ObjectId('64239cb15e6e664c5b59ccb1'),
 'geocode': {'latitude': 51.08084, 'longitude': 1.188537}}
{'BusinessName': 'The Ship',
 '_id': ObjectId('64239cb15e6e664c5b59ccb3'),
 'geocode': {'latitude': 51.08084, 'longitude': 1.188

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

